# Closure Plot Generator

**Version:** 13 April 2025  
**Purpose:** Generate closure plots for 8-bit number systems using Stillwater Universal.

---

## Overview

This utility uses the `generateClosurePlots.hpp` API from the Stillwater Universal library to generate closure plots for three 8-bit number systems:

- `cfloat<8,4>`
- `posit<8,0>`
- `lns<8,3>`

## HOWTO

Before executing the commands in this notebook, you need to generate the closure plots' csv/txt files from the `closure_of_8bit_systems.cpp` executable.


Do this now from your `universal/build` directory.  If you have not generated the build directory see the `build instructions` below.


Once your build directory is generated, from the directory execute 
```bash
playground/play_closure_of_8bit_systems
```


Then, run all files in this notebook to generate the closure plots pngs.

## Output

The following files will be generated in the `build/mappings/user_generated/` directory:

- `cfloat_8_4/` 
    - `cfloat_8_4.csv`
    - `cfloat_8_4.txt`
    - `closure_plot_add.png`
    - `closure_plot_div.png`
    - `closure_plot_mul.png`
    - `closure_plot_sub.png`

- `lns_8_3/` 
    - `lns_8_3.csv`
    - `lns_8_3.txt`
    - `closure_plot_add.png`
    - `closure_plot_div.png`
    - `closure_plot_mul.png`
    - `closure_plot_sub.png`

- `posit_8_0/` 
    - `posit_8_0.csv`
    - `posit_8_0.txt`
    - `closure_plot_add.png`
    - `closure_plot_div.png`
    - `closure_plot_mul.png`
    - `closure_plot_sub.png`

## Build Instructions

To build and run the closure plot generator:

```bash
mkdir -p build && cd build
cmake ..
```



In [76]:
def plot_df(df, system_name, operation):
    import matplotlib.pyplot as plt
    import pandas as pd

    color_map = {
        "Exact": "black",
        "Underflow": "red",
        "Overflow": "red",
        "NAN/NAR": "yellow",
        "Saturate": "green",
        "Approximation": "purple" 
    }

    operation_map = {
        "add": "Addition Plot",
        "sub": "Subtraction Plot", 
        "mul": "Multiplication Plot",
        "div": "Division Plot"
    } 


    operation = operation_map[operation]
    system_name += f": {operation}"

    df_subset = df.iloc[:, [0, 1, 3]].copy()
    df_subset.columns = ["Result", "Value1", "Value2"]

    df_subset["Result"] = df_subset["Result"].astype(str).str.strip()
    df_subset["Value1"] = pd.to_numeric(df_subset["Value1"], errors="coerce")
    df_subset["Value2"] = pd.to_numeric(df_subset["Value2"], errors="coerce")
    df_subset.dropna(subset=["Value1", "Value2"], inplace=True)

    unique_val1 = pd.unique(df_subset["Value1"])
    unique_val2 = pd.unique(df_subset["Value2"])

    val1_to_x = {val: idx for idx, val in enumerate(unique_val1)}
    val2_to_y = {val: idx for idx, val in enumerate(unique_val2)}

    x_coords, y_coords, colors = [], [], []

    for _, row in df_subset.iterrows():
        x = val1_to_x.get(row["Value1"])
        y = val2_to_y.get(row["Value2"])
        result = row["Result"]

        if x is not None and y is not None:
            x_coords.append(x)
            y_coords.append(y)
            colors.append(color_map.get(result, "pink"))

    plt.figure(figsize=(10, 10))
    plt.scatter(x_coords, y_coords, c=colors, s=5, marker='s')

    # Set x/y tick labels: only show -maxreal and maxreal
    x_ticks = list(range(len(unique_val1)))
    y_ticks = list(range(len(unique_val2)))
    x_labels = [''] * len(x_ticks)
    y_labels = [''] * len(y_ticks)
    if x_ticks:
        x_labels[0] = '-maxreal'
        x_labels[-1] = 'maxreal'
        x_labels[len(x_ticks)//2] = 'zero'
    if y_ticks:
        y_labels[0] = '-maxreal'
        y_labels[-1] = 'maxreal'
        y_labels[len(y_ticks)//2] = 'zero'


    x_mid = len(x_ticks) // 2
    y_mid = len(y_ticks) // 2

    plt.xticks(
        ticks=[0, x_mid, len(x_ticks) - 1],
        labels=['-maxreal', 'zero', 'maxreal']
    )
    plt.yticks(
        ticks=[0, y_mid, len(y_ticks) - 1],
        labels=['-maxreal', 'zero', 'maxreal']
    )


    plt.title(system_name)
    plt.grid(False)
    plt.tight_layout()
    plt.show()


In [77]:
import os

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches

import pandas as pd
import numpy as np





def plot_df_grad(df, system_name, operation, render_plot=True, save_dir=None):
    # Define color mapping (no Approximation here to use colormap)

    color_map = {
        "Exact": "black",
        "Underflow": "blue",
        "Overflow": "red",
        "NAR/NAN": "yellow",
        "Saturate": "green"
    }

    operation_map = {
        "add": "Addition Plot",
        "sub": "Subtraction Plot",
        "mul": "Multiplication Plot",
        "div": "Division Plot"
    }

    # Set title
    operation_title = operation_map.get(operation, operation)
    system_name += f": {operation_title}"

    # Copy and clean DataFrame
    df_subset = df.iloc[:, [0, 1, 11, 3]].copy()  # Index 11 for Normalized Relative Log Error
    df_subset.columns = ["Result", "Value1", "Normalized Relative Log Error", "Value2"]
    df_subset["Result"] = df_subset["Result"].astype(str).str.strip()
    df_subset["Value1"] = pd.to_numeric(df_subset["Value1"], errors="coerce")
    df_subset["Value2"] = pd.to_numeric(df_subset["Value2"], errors="coerce")
    df_subset["Normalized Relative Log Error"] = pd.to_numeric(
        df_subset["Normalized Relative Log Error"], errors="coerce"
    )
    df_subset.dropna(subset=["Value1", "Value2"], inplace=True)

    # Remove duplicates to prevent overplotting
    df_subset = df_subset.drop_duplicates(subset=["Value1", "Value2", "Result"])

    # Debug: Print data stats
    print("Unique Value1 count:", len(pd.unique(df_subset["Value1"])))
    print("Unique Value2 count:", len(pd.unique(df_subset["Value2"])))
    print("Total points:", len(df_subset))

    # Set up coordinate mapping
    unique_val1 = pd.unique(df_subset["Value1"])
    unique_val2 = pd.unique(df_subset["Value2"])
    val1_to_x = {val: idx for idx, val in enumerate(unique_val1)}
    val2_to_y = {val: idx for idx, val in enumerate(unique_val2)}

    x_coords, y_coords, colors = [], [], []

    # Custom purple colormap with strong contrast
    purple_cmap = mcolors.LinearSegmentedColormap.from_list(
        "custom_purple", ["#4B0082", "#BA55D3"], N=256  # Dark to medium purple
        # "custom_purple", ["#BA55D3", "#4B0082"], N=256  # Medium to dark purple
    )

    # Determine color scale for Approximation
    approx_subset = df_subset[df_subset["Result"] == "Approximation"].copy()
    valid_errors = approx_subset["Normalized Relative Log Error"].dropna()
    valid_errors = valid_errors[valid_errors > 0]

    # Debug: Print error stats
    if not valid_errors.empty:
        print("Approximation errors - Min:", valid_errors.min(), 
              "Max:", valid_errors.max(), 
              "Count:", len(valid_errors))
    else:
        print("No valid Approximation errors found.")

    if not valid_errors.empty:
        vmin = valid_errors.min()
        vmax = valid_errors.max()
        if np.isfinite(vmin) and np.isfinite(vmax) and vmin < vmax:
            norm = mcolors.LogNorm(vmin=vmin, vmax=vmax)
        else:
            norm = mcolors.LogNorm(vmin=1e-6, vmax=1)
            print(f"Warning: Invalid vmin={vmin} or vmax={vmax}, using defaults.")
    else:
        norm = mcolors.LogNorm(vmin=1e-6, vmax=1)
        print("Warning: Using default LogNorm due to no valid errors.")

    # Assign colors and coordinates
    for _, row in df_subset.iterrows():
        x = val1_to_x.get(row["Value1"])
        y = val2_to_y.get(row["Value2"])
        result = row["Result"]

        if x is not None and y is not None:
            x_coords.append(x)
            y_coords.append(y)

            if result == "Approximation":
                error = row["Normalized Relative Log Error"]
                if pd.isna(error) or error <= 0 or not np.isfinite(error):
                    colors.append(purple_cmap(0.1))  # Slightly darker for invalid errors
                else:
                    normalized_value = norm(error)
                    colors.append(purple_cmap(normalized_value))
            else:
                colors.append(color_map.get(result, "pink"))

    # Plotting
    plt.figure(figsize=(5, 5), dpi=200)  # High DPI for precision
    ax = plt.gca()
    ax.set_facecolor('#E0E0E0')  # Medium gray background
    plt.scatter(x_coords, y_coords, c=colors, s=2.5, marker='s', alpha=1.0, edgecolors='none')

    # X/Y tick labels
    x_ticks = list(range(len(unique_val1)))
    y_ticks = list(range(len(unique_val2)))
    x_mid = len(x_ticks) // 2 if len(x_ticks) > 1 else 0
    y_mid = len(y_ticks) // 2 if len(y_ticks) > 1 else 0

    plt.xticks(
        [0, x_mid, len(x_ticks) - 1] if len(x_ticks) > 1 else [0],
        ['-maxreal', 'zero', 'maxreal'] if len(x_ticks) > 1 else ['zero'],
        rotation=45, ha='right'
    )
    plt.yticks(
        [0, y_mid, len(y_ticks) - 1] if len(y_ticks) > 1 else [0],
        ['-maxreal', 'zero', 'maxreal'] if len(y_ticks) > 1 else ['zero']
    ) 

    plt.tick_params(axis='both', which='both', length=0)

    # Tighten axes limits to reduce border gaps
    ax.set_xlim(-0.5, len(unique_val1) - 0.5)
    ax.set_ylim(-0.5, len(unique_val2) - 0.5)

    plt.title(system_name)
    plt.grid(False) 


    legend_colors = {
        "Exact": "black",
        "Underflow": "blue",
        "Overflow": "red",
        "NAR/NAN": "yellow",
        "Saturate": "green",
        "Approximation": "#B695D3"  # Moderately light purple
    }

    patches = [
        mpatches.Patch(color=color, label=label)
        for label, color in legend_colors.items()
    ]

    plt.legend(
        handles=patches,
        loc='lower center',
        bbox_to_anchor=(0.5, -0.2),
        ncol=3,
        fontsize=7,
        frameon=False
    )


    plt.tight_layout()

    if render_plot:
        plt.show()

    if save_dir is not None:

        filename = f"closure_plot_{operation}.png"
        full_path = os.path.join(save_dir, filename)
        plt.savefig(full_path, bbox_inches='tight', dpi=300)


    plt.close()
    


In [78]:
import pandas as pd
import numpy as np

def generate_posit_df(csv_file):
    with open(csv_file, 'r') as file:
        lines = file.readlines()

    header = lines[1].strip().split(',')
    i = 2  # Skip header and first operation line

    op_map = {
        "Generate '+' table:": "add_df",
        "Generate '-' table:": "sub_df",
        "Generate '*' table:": "mul_df",
        "Generate '/' table:": "div_df"
    }

    # Initialize DataFrame variables
    add_df = None
    sub_df = None
    mul_df = None
    div_df = None

    current_op = None
    current_data = []

    # Outer loop: checks for a new operation table
    while i < len(lines):
        line = lines[i].strip()

        # Check if the line starts with any key in op_map
        for key, value in op_map.items():
            if line.startswith(key):
                # Save previous operation data if any
                if current_op is not None and current_data: 
                    if current_op == "div_df":
                        current_data.sort(key=lambda row: (float(row[1]), float(row[3])))
                    df = pd.DataFrame(current_data, columns=header)
                    # Save the previous operation's DataFrame
                    if current_op == "add_df":
                        add_df = df
                    elif current_op == "sub_df":
                        sub_df = df
                    elif current_op == "mul_df":
                        mul_df = df
                    elif current_op == "div_df":
                        div_df = df

                # Start a new operation
                current_op = value
                current_data = []
                i += 1  # Move to the next line after the operation
                break  # Exit the loop and go to the inner loop

        # Inner loop: Process data rows for the current operation
        while i < len(lines) and not any(lines[i].startswith(key) for key in op_map):
            cur_line = lines[i].strip()
            values = cur_line.split(',')

            # Skip invalid rows containing "nar", "nan", "inf", or "-inf"

            if values[1].lower() in {"nar", "nan", "inf", "-inf", "-0"} or values[3].lower() in {"nar", "nan", "inf", "-inf", "-0"}:
                i += 1
                continue

            # Add valid row to the current operation's data
            current_data.append(values)
            i += 1

        # Continue to the next iteration of the outer loop

    # Save the last operation's DataFrame if any
    if current_op is not None and current_data:
        df = pd.DataFrame(current_data, columns=header)
        current_data.sort(key=lambda row: (float(row[1]), float(row[3])))
        if current_op == "add_df":
            add_df = df
        elif current_op == "sub_df":
            sub_df = df
        elif current_op == "mul_df":
            mul_df = df
        elif current_op == "div_df":
            div_df = df

    return add_df, sub_df, mul_df, div_df


In [79]:
import pandas as pd

def generate_cfloat_df(csv_file):
    with open(csv_file, 'r') as file:
        lines = file.readlines()

    header = lines[1].strip().split(',')
    i = 2  # Skip header and first operation line

    op_map = {
        "Generate '+' table:": "add_df",
        "Generate '-' table:": "sub_df",
        "Generate '*' table:": "mul_df",
        "Generate '/' table:": "div_df"
    }

    # Initialize DataFrame variables
    add_df = sub_df = mul_df = div_df = None

    current_op = None
    current_data = []

    def store_dataframe(op, data):
        # Sort all values from maxpos → maxneg
        try:
            data.sort(key=lambda row: (float(row[1]), float(row[3])), reverse=True)
        except ValueError:
            pass  # Skip sort if conversion fails
        df = pd.DataFrame(data, columns=header)
        return df

    while i < len(lines):
        line = lines[i].strip()

        # Check if this is a new operation block
        for key, value in op_map.items():
            if line.startswith(key):
                # Store the current operation's DataFrame
                if current_op is not None and current_data:
                    df = store_dataframe(current_op, current_data)
                    if current_op == "add_df":
                        add_df = df
                    elif current_op == "sub_df":
                        sub_df = df
                    elif current_op == "mul_df":
                        mul_df = df
                    elif current_op == "div_df":
                        div_df = df

                current_op = value
                current_data = []
                i += 1
                break

        # Gather data rows for the current operation
        while i < len(lines) and not any(lines[i].startswith(key) for key in op_map):
            cur_line = lines[i].strip()
            values = cur_line.split(',')

            # Skip invalid float rows
            if values[1].lower() in {"nar", "nan", "inf", "-inf", "-0"} or \
               values[3].lower() in {"nar", "nan", "inf", "-inf", "-0"}:
                i += 1
                continue

            current_data.append(values)
            i += 1

    # Store the final operation's DataFrame
    if current_op is not None and current_data:
        df = store_dataframe(current_op, current_data)
        if current_op == "add_df":
            add_df = df
        elif current_op == "sub_df":
            sub_df = df
        elif current_op == "mul_df":
            mul_df = df
        elif current_op == "div_df":
            div_df = df

    return add_df, sub_df, mul_df, div_df


In [80]:
import os

POSIT_NAME = f"posit_8_0"
CFLOAT_NAME = f"cfloat_8_4"

SRC_DIR = "./build/mappings/user_generated"

posit_mappings_path = os.path.join(SRC_DIR, f"{POSIT_NAME}/{POSIT_NAME}.csv")
cfloat_mappings_path = os.path.join(SRC_DIR, f"{CFLOAT_NAME}/{CFLOAT_NAME}.csv")

posit_out_path = os.path.join(SRC_DIR, f"{POSIT_NAME}")
cfloat_out_path = os.path.join(SRC_DIR, f"{CFLOAT_NAME}")




''' Generate plots for the posit'''
add_df1, sub_df1, mul_df1, div_df1 = generate_posit_df(posit_mappings_path) 
plot_df_grad(add_df1, POSIT_NAME, "add", render_plot=False, save_dir=posit_out_path)
plot_df_grad(sub_df1, POSIT_NAME, "sub", render_plot=False, save_dir=posit_out_path)
plot_df_grad(mul_df1, POSIT_NAME, "mul", render_plot=False, save_dir=posit_out_path)
plot_df_grad(div_df1, POSIT_NAME, "div", render_plot=False, save_dir=posit_out_path)

# add_df1.to_csv("add_table_1.csv", index=False)
# sub_df1.to_csv("sub_table_1.csv", index=False)
# mul_df1.to_csv("mul_table_1.csv", index=False)
# div_df1.to_csv("div_table_1.csv", index=False) 



''' Generate plots for the cfloat'''
add_df2, sub_df2, mul_df2, div_df2 = generate_cfloat_df(posit_mappings_path) 
plot_df_grad(add_df2, CFLOAT_NAME, "add", render_plot=False, save_dir=cfloat_out_path)
plot_df_grad(sub_df2, CFLOAT_NAME, "sub", render_plot=False, save_dir=cfloat_out_path)
plot_df_grad(mul_df2, CFLOAT_NAME, "mul", render_plot=False, save_dir=cfloat_out_path)
plot_df_grad(div_df2, CFLOAT_NAME, "div", render_plot=False, save_dir=cfloat_out_path)

# add_df2.to_csv("add_table_2.csv", index=False)
# sub_df2.to_csv("sub_table_2.csv", index=False)
# mul_df2.to_csv("mul_table_2.csv", index=False)
# div_df2.to_csv("div_table_2.csv", index=False) 



Unique Value1 count: 255
Unique Value2 count: 255
Total points: 65025
Approximation errors - Min: 2.93553e-05 Max: 0.0487469 Count: 16899
Unique Value1 count: 255
Unique Value2 count: 255
Total points: 65025
Approximation errors - Min: 2.93553e-05 Max: 0.0487469 Count: 16899
Unique Value1 count: 255
Unique Value2 count: 255
Total points: 65025
Approximation errors - Min: 3.02947e-05 Max: 0.0487469 Count: 27644
Unique Value1 count: 255
Unique Value2 count: 255
Total points: 65025
Approximation errors - Min: 3.02947e-05 Max: 0.0487469 Count: 27660
Unique Value1 count: 255
Unique Value2 count: 255
Total points: 65025
Approximation errors - Min: 2.93553e-05 Max: 0.0487469 Count: 16899
Unique Value1 count: 255
Unique Value2 count: 255
Total points: 65025
Approximation errors - Min: 2.93553e-05 Max: 0.0487469 Count: 16899
Unique Value1 count: 255
Unique Value2 count: 255
Total points: 65025
Approximation errors - Min: 3.02947e-05 Max: 0.0487469 Count: 27644
Unique Value1 count: 255
Unique Va